In [22]:
from pyexpat.errors import messages

from dotenv import load_dotenv
import os

load_dotenv(override=True)
API_KEY = os.getenv('APCA-API-KEY-ID')
SECRET_KEY = os.getenv('APCA-API-SECRET-KEY')

In [23]:
import asyncio, websockets, json, nest_asyncio
nest_asyncio.apply()

In [24]:
url = "wss://stream.data.alpaca.markets/v1beta3/crypto/us"
# url = "wss://stream.data.sandbox.alpaca.markets/v1beta3/crypto/us"
# url = "wss://stream.data.alpaca.markets/v2/test"

In [25]:
async def connect_to_websocket():
    async with websockets.connect(url) as connected_websocket:
        msg = await connected_websocket.recv()
        print(msg)     
asyncio.get_event_loop().run_until_complete(connect_to_websocket())

[{"T":"success","msg":"connected"}]


In [26]:
auth_data = {
        "action": "auth",
        "key": API_KEY, 
        "secret": SECRET_KEY
}

In [30]:
async def authenticate_websocket():
    async with (websockets.connect(url) as ws):
        await ws.send(json.dumps(auth_data))
        async for msg in ws:
            message_data = json.loads(msg)
            if message_data[0]["msg"] == "connected":
                print(message_data)
            elif message_data[0]["msg"] == "authenticated":
                print(message_data)
                break
            else:
                print("error", message_data)
                break
        return ws
ws = asyncio.get_event_loop().run_until_complete(authenticate_websocket())

[{'T': 'success', 'msg': 'connected'}]
[{'T': 'success', 'msg': 'authenticated'}]


In [5]:
# trading - wss://paper-api.alpaca.markets/stream
# market - wss://stream.data.alpaca.markets/v1beta3/crypto/us

In [31]:
async def subscribe_to_ticker(symbol):
    async with ws:
        sub_message = {
            "action": "subscribe",
            "bars": [symbol],
        }
        await ws.send(json.dumps(sub_message))
        async for msg in ws:
            message_data = json.loads(msg)
            print(message_data)
asyncio.get_event_loop().run_until_complete(subscribe_to_ticker("BTC/USD"))

TypeError: 'WebSocketClientProtocol' object does not support the asynchronous context manager protocol

In [9]:
def on_open(ws):
    print("opened")
    auth_data = {
        "action": "authenticate",
        "key_id": API_KEY, 
        "secret_key": SECRET_KEY
    }

    ws.send(json.dumps(auth_data))

    listen_message = {"action": "listen", "data": {"streams": ["AM.TSLA"]}}

    ws.send(json.dumps(listen_message))
# 
# 
def on_message(ws, message):
    print("received a message")
    print(message)

def on_close(ws):
    print("closed connection")
# 
# socket = "wss://stream.data.alpaca.markets/v1beta3/crypto/us"
# 
ws = websocket.WebSocketApp(socket, on_open=on_open, on_message=on_message, on_close=on_close)
# ws.run_forever()

NameError: name 'socket' is not defined